In [1]:
import os 

os.chdir('/work/sghoshstat/ywu39393/AUTOSurv/autostat')

In [2]:
import pandas as pd

In [3]:
data_t = pd.read_csv('train_test_split/minmax_normalized/data_train_minmax_overall.csv')
data_t.shape

(857, 3222)

In [4]:
data_test = pd.read_csv('train_test_split/minmax_normalized/data_test_minmax_overall.csv')
data_test.shape

(201, 3222)

In [5]:
data = pd.concat([data_t, data_test], ignore_index=True)
data.shape

(1058, 3222)

In [6]:
data_gene = pd.read_csv('train_test_split/minmax_normalized/data_train_gene_minmax_overall.csv')
data_gene.shape

(857, 2706)

In [7]:
data_mirna = pd.read_csv('train_test_split/minmax_normalized/data_train_mirna_minmax_overall.csv')
data_mirna.shape

(857, 523)

# elastic net regression

In [8]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sksurv.metrics import concordance_index_censored
from sklearn.linear_model import ElasticNetCV


In [9]:


# Assuming 'time' and 'event' are the columns to be excluded from covariates
X = data.iloc[:,7:]
y = data.iloc[:,2]
X.head(5)

,hsa.let.7d,hsa.let.7f.1,hsa.let.7f.2,hsa.let.7g,hsa.mir.100,hsa.mir.101.1,hsa.mir.101.2,hsa.mir.103a.1,hsa.mir.103a.2,hsa.mir.105.1,...,ENSG00000275302,ENSG00000275713,ENSG00000275832,ENSG00000276231,ENSG00000276966,ENSG00000277161,ENSG00000277494,ENSG00000277632,ENSG00000278677,ENSG00000278828
0,0.499320,0.437412,0.437041,0.311204,0.492384,0.242038,0.231043,0.513902,0.513657,0.450414,...,0.406375,0.515713,0.461430,0.344072,0.316841,0.581027,0.136593,0.309727,0.619096,0.736450
1,0.551029,0.568786,0.572681,0.475083,0.464769,0.436789,0.431574,0.750585,0.750376,0.000000,...,0.358977,0.493683,0.408876,0.433838,0.454092,0.589997,0.190898,0.184881,0.327832,0.518980
2,0.383912,0.432159,0.433002,0.315988,0.683195,0.550232,0.547749,0.612225,0.611744,0.000000,...,0.484220,0.009179,0.407113,0.299033,0.067172,0.586501,0.093736,0.584214,0.104556,0.154693
3,0.398609,0.489787,0.489789,0.294478,0.636696,0.542641,0.534874,0.569820,0.567907,0.020774,...,0.308629,0.082920,0.414558,0.424757,0.021270,0.519437,0.158639,0.248263,0.011822,0.079747
4,0.485919,0.474499,0.477889,0.550921,0.584468,0.574000,0.565838,0.594480,0.595835,0.033543,...,0.202231,0.403341,0.053372,0.214786,0.370362,0.667011,0.068821,0.512853,0.469517,0.761500


In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled.shape

(1058, 3215)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Get the optimal parameters

In [12]:
model = ElasticNetCV(
    l1_ratio=[.1, .5, .7, .9, 1],  # Range of l1_ratio values to try
    alphas=np.logspace(-2, 0, 5),  # Range of alpha values to try
    cv=5,  # Number of cross-validation folds  
    random_state=42
)
model.fit(X_train, y_train)

/common/sghoshstat/ywu39393/DKL/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.395e+08, tolerance: 8.095e+04
  model = cd_fast.enet_coordinate_descent(
/common/sghoshstat/ywu39393/DKL/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.032e+07, tolerance: 8.095e+04
  model = cd_fast.enet_coordinate_descent(
/common/sghoshstat/ywu39393/DKL/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

ElasticNetCV(alphas=array([0.01      , 0.03162278, 0.1       , 0.31622777, 1.        ]),
             cv=5, l1_ratio=[0.1, 0.5, 0.7, 0.9, 1], random_state=42)

In [14]:
# Optimal alpha and l1_ratio
optimal_alpha = model.alpha_
optimal_l1_ratio = model.l1_ratio_

print(f"Optimal alpha: {optimal_alpha}")
print(f"Optimal l1_ratio: {optimal_l1_ratio}")



Optimal alpha: 1.0
Optimal l1_ratio: 0.1


In [15]:


# Model coefficients
coef = model.coef_

# Identify features with non-zero coefficients
important_features = np.where(coef != 0)[0]

# Get the names of the important features
feature_names = data.drop(columns=['OS', 'OS.time']).columns
selected_features = feature_names[important_features]
print(f"Selected features: {selected_features}")

# # Transform the dataset to include only the selected features
# X_reduced = X[:, important_features]

# # Create a new DataFrame with the selected features
# reduced_data = pd.DataFrame(X_reduced, columns=selected_features)
# print(reduced_data.head())


Selected features: Index(['patient_id', 'age', 'race_white', 'stage_i', 'stage_ii', 'hsa.let.7d',
       'hsa.let.7f.1', 'hsa.let.7f.2', 'hsa.let.7g', 'hsa.mir.100',
       ...
       'ENSG00000271503', 'ENSG00000272047', 'ENSG00000272886',
       'ENSG00000273542', 'ENSG00000273841', 'ENSG00000275302',
       'ENSG00000275713', 'ENSG00000275832', 'ENSG00000276231',
       'ENSG00000276966'],
      dtype='object', length=3182)


## Using fixed alpha and l1 ratio

In [20]:
model = ElasticNet(alpha=1.0, l1_ratio=0.5)  # Adjust alpha and l1_ratio as needed
model.fit(X_train, y_train)

/common/sghoshstat/ywu39393/DKL/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.904e+07, tolerance: 1.064e+05
  model = cd_fast.enet_coordinate_descent(


ElasticNet()

In [21]:
coef = model.coef_
coef

array([ -4.90816285,  -6.07298976,  -6.64032365, ..., -42.72655467,
         2.54392723,   1.49050729])

In [22]:
important_features = np.where(coef != 0)[0]
important_features

array([   0,    1,    2, ..., 3212, 3213, 3214])

In [26]:
feature_names = X.columns
selected_features = feature_names[important_features]
selected_features.shape


(2985,)